In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

# 设置随机种子保证可重复性
torch.manual_seed(42)
np.random.seed(42)

class TransformerSurvival(nn.Module):
    def __init__(self, in_features, d_model=32, out_features=1, 
                 nhead=4, num_layers=2, dropout=0.1, batch_norm=True, output_bias=False):
        super().__init__()
        
        # Input projection - 将输入特征投影到Transformer的维度
        self.input_proj = nn.Linear(in_features, d_model)
        
        # Positional encoding - 位置编码
        self.pos_encoding = nn.Parameter(torch.randn(1, 1, d_model))
        
        # Transformer encoder
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=d_model * 4,
            dropout=dropout,
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers)
        
        # Batch norm
        self.bn = nn.BatchNorm1d(d_model) if batch_norm else nn.Identity()
        
        # 直接输出层 - 从Transformer维度映射到输出维度
        self.output_layer = nn.Linear(d_model, out_features, bias=output_bias)
        
        self.dropout = nn.Dropout(dropout)
        self.d_model = d_model

    def forward(self, x, verbose=False):
        if verbose:
            print(f"输入 x: {x.shape}, 数值范围: [{x.min():.4f}, {x.max():.4f}]")
            if x.shape[0] <= 3:  # 只对小batch显示具体数值
                print(f"输入数值:\n{x}")
        
        # Input processing
        x = self.input_proj(x)
        if verbose:
            print(f"\n1. 输入投影后: {x.shape}")
            print(f"   input_proj.weight: {self.input_proj.weight.shape}")
            print(f"   input_proj.bias: {self.input_proj.bias.shape}")
            if x.shape[0] <= 3:
                print(f"   数值:\n{x}")
        
        x = self.bn(x)
        if verbose:
            print(f"\n2. BatchNorm后: {x.shape}")
            if hasattr(self.bn, 'weight'):
                print(f"   bn.weight: {self.bn.weight.shape}")
                print(f"   bn.bias: {self.bn.bias.shape}")
            if x.shape[0] <= 3:
                print(f"   数值:\n{x}")
        
        x = F.relu(x)
        if verbose:
            print(f"\n3. ReLU后: {x.shape}")
            print(f"   正值比例: {(x > 0).float().mean():.2%}")
            if x.shape[0] <= 3:
                print(f"   数值:\n{x}")
        
        x = self.dropout(x)
        if verbose:
            print(f"\n4. Dropout后: {x.shape}")
            print(f"   零值比例: {(x == 0).float().mean():.2%}")
            if x.shape[0] <= 3:
                print(f"   数值:\n{x}")
        
        # Transformer processing
        x = x.unsqueeze(1)
        if verbose:
            print(f"\n5. 添加序列维度后: {x.shape}")
            if x.shape[0] <= 3:
                print(f"   数值:\n{x}")
        
        x = x + self.pos_encoding
        if verbose:
            print(f"\n6. 添加位置编码后: {x.shape}")
            print(f"   pos_encoding: {self.pos_encoding.shape}")
            if x.shape[0] <= 3:
                print(f"   位置编码数值:\n{self.pos_encoding}")
                print(f"   相加后数值:\n{x}")
        
        x = self.transformer(x)
        if verbose:
            print(f"\n7. Transformer处理后: {x.shape}")
            print(f"   Transformer层数: {len(self.transformer.layers)}")
            if x.shape[0] <= 3:
                print(f"   数值:\n{x}")
        
        x = x.squeeze(1)
        if verbose:
            print(f"\n8. 移除序列维度后: {x.shape}")
            if x.shape[0] <= 3:
                print(f"   数值:\n{x}")
        
        # 直接映射到输出
        output = self.output_layer(x)
        if verbose:
            print(f"\n9. 输出层处理后: {output.shape}")
            print(f"   output_layer.weight: {self.output_layer.weight.shape}")
            if self.output_layer.bias is not None:
                print(f"   output_layer.bias: {self.output_layer.bias.shape}")
            print(f"   最终输出数值:\n{output}")
        
        return output

# 创建演示数据
def create_demo_data(batch_size=2, in_features=10):
    """创建演示用的随机数据"""
    # 创建一些有意义的特征数据
    np.random.seed(42)
    
    # 基础特征：年龄、性别、各种生物标志物等
    data = np.random.randn(batch_size, in_features)
    
    # 让数据更有意义一些
    data[:, 0] = np.random.normal(60, 15, batch_size)  # 年龄 ~ N(60, 15)
    data[:, 1] = np.random.randint(0, 2, batch_size)   # 性别 (0/1)
    
    # 其他特征标准化到合理范围
    for i in range(2, in_features):
        data[:, i] = np.random.normal(0, 1, batch_size)
    
    return torch.FloatTensor(data)

# 演示代码
def demo_transformer_survival():
    print("=" * 60)
    print("TransformerSurvival 模型演示")
    print("=" * 60)
    
    # 参数设置
    batch_size = 2
    in_features = 4
    d_model = 32
    out_features = 1
    
    # 创建模型
    model = TransformerSurvival(
        in_features=in_features,
        d_model=d_model,
        out_features=out_features,
        nhead=4,
        num_layers=2,
        dropout=0.1,
        batch_norm=True,
        output_bias=True
    )
    
    # 创建演示数据
    demo_input = create_demo_data(batch_size, in_features)
    
    print(f"\n模型参数:")
    print(f"- 输入特征数: {in_features}")
    print(f"- d_model: {d_model}")
    print(f"- 输出特征数: {out_features}")
    print(f"- 多头注意力头数: 4")
    print(f"- Transformer层数: 2")
    
    print(f"\n输入数据形状: {demo_input.shape}")
    print(f"输入数据示例:")
    print(demo_input)
    
    print(f"\n{'='*60}")
    print("开始前向传播过程...")
    print(f"{'='*60}")
    
    # 进行前向传播并显示详细信息
    with torch.no_grad():  # 不计算梯度，只用于演示
        output = model(demo_input, verbose=True)
    
    print(f"\n{'='*60}")
    print("模型总结:")
    print(f"{'='*60}")
    
    # 计算参数量
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    print(f"总参数量: {total_params:,}")
    print(f"可训练参数量: {trainable_params:,}")
    
    # 各层参数量统计
    print(f"\n各层参数量:")
    for name, param in model.named_parameters():
        print(f"  {name}: {param.numel():,} parameters")
    
    print(f"\n最终输出形状: {output.shape}")
    print(f"最终输出值: {output.squeeze().tolist()}")

# 运行演示
if __name__ == "__main__":
    demo_transformer_survival()

TransformerSurvival 模型演示

模型参数:
- 输入特征数: 4
- d_model: 32
- 输出特征数: 1
- 多头注意力头数: 4
- Transformer层数: 2

输入数据形状: torch.Size([2, 4])
输入数据示例:
tensor([[52.9579,  1.0000, -0.5714, -2.6125],
        [68.1384,  1.0000, -0.9241,  0.9504]])

开始前向传播过程...
输入 x: torch.Size([2, 4]), 数值范围: [-2.6125, 68.1384]
输入数值:
tensor([[52.9579,  1.0000, -0.5714, -2.6125],
        [68.1384,  1.0000, -0.9241,  0.9504]])

1. 输入投影后: torch.Size([2, 32])
   input_proj.weight: torch.Size([32, 4])
   input_proj.bias: torch.Size([32])
   数值:
tensor([[ 19.9454,  -6.4280,  22.9129,  19.8331,  19.8632, -12.6629, -20.4037,
           2.6653,  19.3962,   4.4144,   5.8354,  14.5800,  14.1243, -20.9334,
           9.4413,   1.3207,   8.4604, -16.6780,  26.6302, -18.3936, -13.1834,
          16.4071, -12.8117,  16.8127,  25.3875, -14.4271,   1.6967, -24.3923,
           8.5831,   4.7814,   0.9548, -17.6082],
        [ 27.4262,  -6.9590,  29.7842,  25.1042,  26.2508, -14.9067, -27.4165,
           1.7091,  26.4142,   5.0162,   9.391